# Regression problem using RBF

In [31]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras

from keras.datasets import boston_housing
from keras import Model, layers
from keras.metrics import RootMeanSquaredError
from keras import backend as K

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

import numpy as np
from scipy.spatial.distance import cdist

import time
import sys

import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
# The radial basis function layer
def rbf_layer(x, c):
    rbf_neurons = np.shape(c)[0]

    # Find sigma
    d_max = cdist(c, c, metric="euclidean").max()
    sigma = d_max / np.sqrt(2 * rbf_neurons)

    # Stores the transformed set x after it passes the rbf layer
    transformed_x = np.zeros([np.shape(x)[0], rbf_neurons], dtype=np.float32)

    # Pass the x set through every rbf neuron
    for i, center in enumerate(c):
        # Find the squared norms
        norms = np.linalg.norm((x - center), axis=1) ** 2

        transformed_x[:, i] = np.exp(-norms) / (2 * sigma**2)

    return transformed_x

In [33]:
# Function that plots figures based on the history of the training
def loss_acc_plot(history):
    plt.figure()
    plt.plot(history.history['coeff_determination'], label = 'Train')
    plt.plot(history.history['val_coeff_determination'], label = 'Validation')
    plt.legend()
    plt.title('R squared of train and validation sets')

    plt.figure()
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.legend()
    plt.title('Loss of train and validation sets')

    plt.show()

In [34]:
# Function for R squared
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

Data Preprocessing

In [35]:
# Preparation of MNIST data
(x_train, y_train), (x_test, y_test) = boston_housing.load_data(test_split=0.25)

# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
y_train, y_test = np.array(y_train, np.float32), np.array(y_test, np.float32)

# Scale the training and testing data
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [36]:
# Number of RBF neurons is the 10% of the training data
rbf_neurons = int(0.1*np.shape(y_train)[0])

kmeans = KMeans(n_clusters=rbf_neurons).fit(x_train)
centers = kmeans.cluster_centers_

# Pass the train set from the rbf layer
x_train_transformed = rbf_layer(x_train, centers)

print(x_train_transformed[:5])

[[3.21814862e-12 6.84783809e-05 6.52635836e-19 2.69383600e-04
  8.86917942e-17 2.40678805e-14 1.55371577e-01 1.95273615e-12
  5.37155991e-08 1.08066465e-15 2.91963929e-08 1.91229658e-04
  5.63834910e-04 1.03767300e-29 6.58876104e-07 3.18121307e-09
  3.27753935e-10 3.77742499e-12 8.73209838e-12 6.29532030e-14
  1.23170717e-02 7.79954577e-03 2.20279071e-05 2.03780615e-07
  6.39803835e-12 2.60959310e-12 2.89230462e-04 6.49284305e-17
  2.43878079e-13 3.51148299e-39 1.66828595e-09 1.06152154e-17
  9.80550041e-09 9.39416568e-05 4.15567082e-29 2.90279404e-05
  3.84300165e-08]
 [3.05460874e-35 7.02716330e-09 5.88037031e-35 1.34796449e-23
  9.37313065e-02 1.82312929e-33 3.81729212e-18 3.81598850e-11
  2.68401849e-18 1.09594044e-14 3.51228146e-27 4.24925286e-12
  5.41908142e-12 0.00000000e+00 6.26640240e-10 8.25311290e-04
  3.64472999e-22 2.00848845e-34 7.58621148e-32 2.67552623e-31
  2.46882867e-16 6.43544666e-14 1.69337197e-24 6.95127511e-08
  3.93181062e-06 8.66795657e-04 4.90684958e-11 1.457

In [37]:
model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(rbf_neurons,), activation='relu'),
    keras.layers.Dense(1)
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               4864      
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 4,993
Trainable params: 4,993
Non-trainable params: 0
_________________________________________________________________


In [38]:
optim = keras.optimizers.SGD(learning_rate=0.001)
loss_func = keras.losses.MeanSquaredError()

model.compile(optimizer=optim, loss=loss_func, metrics=[coeff_determination, RootMeanSquaredError()])

st = time.time()

history = model.fit(x_train_transformed, y_train, epochs=400, validation_split=0.2)

end = time.time()

print('Training time:', end-st, 'seconds')

Epoch 1/400
10/10 [==============================] - 1s 42ms/step - loss: 536.2168 - coeff_determination: -6.3457 - root_mean_squared_error: 23.1564 - val_loss: 724.3884 - val_coeff_determination: -7.5512 - val_root_mean_squared_error: 26.9145
Epoch 2/400
10/10 [==============================] - 0s 15ms/step - loss: 492.8192 - coeff_determination: -6.0549 - root_mean_squared_error: 22.1995 - val_loss: 665.1826 - val_coeff_determination: -6.7679 - val_root_mean_squared_error: 25.7911
Epoch 3/400
10/10 [==============================] - 0s 14ms/step - loss: 434.9948 - coeff_determination: -5.5879 - root_mean_squared_error: 20.8565 - val_loss: 578.9367 - val_coeff_determination: -5.6333 - val_root_mean_squared_error: 24.0611
Epoch 4/400
10/10 [==============================] - 0s 13ms/step - loss: 352.3611 - coeff_determination: -4.3224 - root_mean_squared_error: 18.7713 - val_loss: 459.8407 - val_coeff_determination: -4.0831 - val_root_mean_squared_error: 21.4439
Epoch 5/400
10/10 [=====